# Init

In [1]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

# Craft Prompt

In [5]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
version = 'v1.0.0'
fname = f'sql_generator-stage_1-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())

def make_prompt(data: dict) -> str:

    prompt = prompt_template
    print(id(prompt) == id(prompt_template))

    # 
    table_finder_res = data['table_finder']['stage_1'][0]['data_source'][0]
    del table_finder_res['question']
    table = table_finder_res['table']
    table_finder_res = json.dumps(table_finder_res, ensure_ascii=False, indent=2)
    reg_p = re.compile('<Database and Table>')
    prompt = re.sub(reg_p, table_finder_res, prompt)

    # 

    table_fname = f'{table}-with_table_name.md'
    table_dir = os.path.join(cwd, 'data' + os.sep + 'table-column')
    table_fpath = os.path.join(table_dir, table_fname)
    with open(table_fpath,'r') as f:
        table_schema = ''.join(f.readlines())
    reg_p = re.compile('<Table-Column Schema>')
    prompt = re.sub(reg_p, table_schema, prompt)

    # 
    ner_res = [i for i in data['ner']['stage_1']['sql'].values() if i][0]
    ner_res = json.dumps(ner_res, ensure_ascii=False, indent=2)
    reg_p = re.compile('<Background Knowledge>')
    prompt = re.sub(reg_p, ner_res, prompt)

    # replace query
    query = data['team'][0]['question']
    reg_p = re.compile('<Current Query>')
    prompt = re.sub(reg_p, query, prompt)

    return prompt

In [3]:
t = {
    "tid": "tttt----1",
    "team": [
        {
            "id": "tttt----1----1-1-1",
            "question": "600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？"
        },
        {
            "id": "tttt----1----1-1-2",
            "question": "该公司实控人是否发生改变？如果发生变化，什么时候变成了谁？是哪国人？是否有永久境外居留权？（回答时间用XXXX-XX-XX）"
        },
        {
            "id": "tttt----1----1-1-3",
            "question": "在实控人发生变化的当年股权发生了几次转让？"
        }
    ],
    "ner": {
        "stage_1": {
            "reasoning_process_cot": "根据查询内容，'600872' 是一个股票代码，指向了一个上市公司，因此应该识别为一个代码。而'全称'、'A股简称'、'法人'、'法律顾问'、'会计师事务所'及'董秘'等词汇虽然出现在查询中，但它们并不构成独立的实体，而是与'600872'相关的属性或角色，因此不需要作为实体识别。所以，我们只需识别'600872'作为代码实体。",
            "result": [
                {
                    "代码": "600872"
                }
            ],
            "sql": {
                "代码:600872": [
                    {
                        "query": "SELECT * FROM ConstantDB.SecuMain WHERE 600872 IN (InnerCode, CompanyCode, SecuCode, ISIN, JSID)",
                        "result": [
                            {
                                "ID": 315934536696,
                                "InnerCode": 2120,
                                "CompanyCode": 1805,
                                "SecuCode": "600872",
                                "ChiName": "中炬高新技术实业(集团)股份有限公司",
                                "ChiNameAbbr": "中炬高新",
                                "EngName": "Jonjee Hi-Tech Industrial And Commercial Holding Co.,Ltd",
                                "EngNameAbbr": "JONJEE",
                                "SecuAbbr": "中炬高新",
                                "ChiSpelling": "ZJGX",
                                "SecuMarket": 83,
                                "SecuCategory": 1,
                                "ListedDate": "1995-01-24 12:00:00.000",
                                "ListedSector": 1,
                                "ListedState": 1,
                                "XGRQ": "2019-08-22 06:42:02.380",
                                "JSID": 619815689135,
                                "ISIN": "CNE000000HK0",
                                "ExtendedAbbr": None,
                                "ExtendedSpelling": None
                            }
                        ]
                    },
                    {
                        "query": "SELECT * FROM ConstantDB.HK_SecuMain WHERE 600872 IN (InnerCode, CompanyCode, SecuCode, ISIN, JSID)",
                        "result": []
                    },
                    {
                        "query": "SELECT * FROM ConstantDB.US_SecuMain WHERE 600872 IN (InnerCode, CompanyCode, SecuCode, ISIN, JSID)",
                        "result": []
                    }
                ]
            }
        }
    },
    "table_finder": {
        "stage_1": [
            {
                "raw_question": "600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？",
                "data_source_reasoning": "这个问题询问的是股票代码为600872的公司的全称、A股简称、法人、法律顾问、会计师事务所及董秘。我们可以这样逐步分析： 1. **识别实体：** 问题中使用了股票代码（600872）明确指出了一个特定的公司，这意味着我们需要与特定上市公司相关的信息。 2. **信息类型：** 问题寻求多种信息：公司全称、A股简称、法人、法律顾问、会计师事务所及董秘。这些都是上市公司的基本属性。 3. **定位数据：** 查看数据库架构，'上市公司基本资料' (AStockBasicInfoDB) 数据库似乎与基本公司信息最为相关。在这个数据库中，'公司概况' (LC_StockArchives) 表被描述为包含上市公司的基本情况，包括联系方式、注册信息、中介机构等。这张表很可能包含问题中要求的具体属性。 4. **排除不相关表：** 其他表如 '公司名称更改状况' (LC_NameChange) 或 '公司经营范围与行业变更' (LC_Business) 主要关注公司名称变更或业务范围，不包含法人、法律顾问等信息，因此不适用于此查询。 5. **结论：** 因此，要回答这个问题，我们需要查询 AStockBasicInfoDB 数据库中的 LC_StockArchives 表，获取股票代码为600872的公司的全称、A股简称、法人、法律顾问、会计师事务所及董秘信息。",
                "data_source": [
                    {
                        "question": "600872的全称、A股简称、法人、法律顾问、会计师事务所及董秘是？",
                        "database": "AStockBasicInfoDB",
                        "table": "LC_StockArchives"
                    }
                ],
                "database_dependency": True
            }
        ]
    },
    "token_usage": {
        "table_finder-stage_1": [
            {
                "prompt_tokens": 9451,
                "completion_tokens": 400,
                "total_tokens": 9851
            }
        ]
    },
    "time_usage": {
        "table_finder-stage_1": [
            "22.85s"
        ]
    }
}

In [4]:
print(make_prompt(t))

True


TypeError: unhashable type: 'list'